## Write by  [Jianguo Zhang](https://github.com/JianguoZhang1994) , April 4, 2017
 Multiple Folders assign labels for acceleration data, 1 label is 1
second, 1 second corresponds to 12 datas, each data includes 3 x, y, z

Original data in **Baboon_Coding** folder and **output** folder. **Baboon_Coding** folder includes several xls files, **output** folder includes several acceleration dataset. Notices some xls files do not have corresponding acceleration data, you need to deal with carefully.

Results in **Feeding** folder, **Interacting** folder and **Multiple_label** folder. For **Feeding**, where 1 represents feeding, 0 represents not feeding. For **Interacting folder**, where 1 represents interacting, 0 represents not interacting. For **Multiple_labels folder**, where 1 is running, 2 is walking, 3 is sitting, 4 is standing at rest, 5 is others. 


In [ ]:
import os.path

folder1='./Baboon_Coding/'
filename1=[]
full_path1=[]
for filename in os.listdir(folder1):
    filename1.append(filename)

"""filename1: xlsfiles; filename2: matfiles"""    
    
folder2='./output/'
filename2=[]
full_path2=[]
for filename in os.listdir(folder2):
    filename2.append(filename)

filename1=filename1[1:]
for i in range(len(filename1)):
    full_path1.append(folder1+filename1[i])

for i in range(len(filename2)):
    full_path2.append(folder2+filename2[i])

full_path1[0]

In [2]:
full_path2[1]

'./output/0808_00001_2426_lau.mat'

In [3]:
folder3='./Feeding/'
folder4='./Interacting/'
folder5='./Multiple_labels/'

In [4]:
filename1[0][0:len(filename1[0])-4]

'0808_00000_2457_Lau'

In [5]:
filename2[0][0:len(filename2[0])-4]

'0808_00000_2457_lau'

In [6]:
import scipy.io as scio
import pandas as pd
import math 
import numpy as np

In [766]:
#Read data
for iteration in range(len(full_path1)): 
   
    #get 0808_00000_2457_Lau
    attribute_file=filename1[iteration][0:len(filename1[iteration])-4]
    f=False
    for index in range(len(filename2)):
        if attribute_file.lower()==filename2[index][0:len(filename2[index])-4].lower():
            f=True
    if(f==False):
        continue


    #dataFile=full_path2[iteration]
    dataFile=folder2+attribute_file+'.mat'
    #dataFile = './output/0808_00000_2457_Lau.mat'
    data = scio.loadmat(dataFile)
    acc_data_x=data['acc_data_x'][0]
    acc_data_y=data['acc_data_y'][0]
    acc_data_z=data['acc_data_z'][0]
    N=len(acc_data_x)
    
    
    xls_path=full_path1[iteration]
    xlsFile=pd.read_excel(xls_path)
    
    time=[]
    subject=[]
    behavior=[]
    status=[]
    df=[]
    
   
    print(attribute_file)
    
    
    df_time=xlsFile['Observation id']
    df_subject=xlsFile[attribute_file]
    df_behavior=xlsFile['Unnamed: 2']
    df_status=xlsFile['Unnamed: 4']

    for ii in range(len(df_time)):
        if(df_time[ii]=='Time'):
            break

    #Remove 'feeding', 'Out of Frame', 'Interacting'        
    for j in range(ii+1, len(df_time)):
        #if(df_behavior[i+1:][j] not in('Feeding', 'Out of Frame', 'Interacting')):
        df.append([float(df_time[j]), df_subject[ii+1:][j], df_behavior[ii+1:][j], df_status[ii+1:][j]])
        time.append(float(df_time[j]))
        subject.append(df_subject[ii+1:][j])
        behavior.append(df_behavior[ii+1:][j])
        status.append(df_status[ii+1:][j])
    """get df"""
    
    ##***********Feeding or not*****************
    labels_feeding=np.zeros(N)
    
    flag=False
    for i in range(len(df)):
        if(behavior[i]=="Feeding" and status[i]=="START"): 
            a=math.floor(time[i])
    #         if(a>int(N/12)):
    #             break
            flag=False
            if(a+1-time[i]>=0.5):
                labels_feeding[a]=1
            else:
                labels_feeding[a]=0
        elif(behavior[i]=="Feeding" and status[i]=="STOP"):
            b=math.floor(time[i])
    #         if(b>int(N/12)):
    #             break
            flag=True
            if(time[i]-b>=0.5):
                labels_feeding[b]=1
            else:
                labels_feeding[b]=0
        if(flag):
            print(a, b)

            for j in range(a+1, b):
                labels_feeding[j]=1
            flag=False
    if(a>b):
        for i in range(a+1, N):
            labels_feeding[i]=1


    labels_feeding=labels_feeding[0:int(N/12)]


    np.savetxt(folder3+attribute_file+'.csv', labels_feeding, fmt='%d', header='Feeding')    

    ###****************Interacting or not******************
    labels_Interacting=np.zeros(N)

    for i in range(len(df)):
        if(behavior[i]=="Interacting" and status[i]=="POINT"): 
            a=math.floor(time[i])
            labels_Interacting[a]=1

    labels_Interacting=labels_Interacting[0:int(N/12)]


    np.savetxt(folder4+attribute_file+'.csv', labels_Interacting, fmt='%d', header='Interacting')    


    
    
    
    
    ###****************Multiple behaviors******************
    labels_multiple=np.ones(N)*5

    flag1=False
    flag2=False
    flag3=False
    flag4=False
    for i in range(len(df)):
        #Running
        if(behavior[i]=="Running" and status[i]=="START"): 
            a1=math.floor(time[i])
            flag1=False
            if(a1+1-time[i]>=0.5):
                labels_multiple[a1]=1       
        elif(behavior[i]=="Running" and status[i]=="STOP"):
            b1=math.floor(time[i])
            flag1=True
            if(time[i]-b1>=0.5):
                labels_multiple[b1]=1 

        #Walking
        if(behavior[i]=="Walking" and status[i]=="START"): 
            a2=math.floor(time[i])
            flag2=False
            if(a2+1-time[i]>=0.5):
                labels_multiple[a2]=2
        elif(behavior[i]=="Walking" and status[i]=="STOP"):
            b2=math.floor(time[i])
            flag2=True
            if(time[i]-b2>=0.5):
                labels_multiple[b2]=2

        #Sitting
        if(behavior[i]=="Sitting" and status[i]=="START"): 
            a3=math.floor(time[i])
            flag3=False
            if(a3+1-time[i]>=0.5):
                labels_multiple[a3]=3
        elif(behavior[i]=="Sitting" and status[i]=="STOP"):
            b3=math.floor(time[i])
            flag3=True
            if(time[i]-b3>=0.5):
                labels_multiple[b3]=3

        #Standing at rest
        if(behavior[i]=="Standing at rest" and status[i]=="START"): 
            a4=math.floor(time[i])
            flag4=False
            if(a4+1-time[i]>=0.5):
                labels_multiple[a4]=4     
        elif(behavior[i]=="Standing at rest" and status[i]=="STOP"):
            b4=math.floor(time[i])
            flag4=True
            if(time[i]-b4>=0.5):
                 labels_multiple[b4]=4

        if(flag1):
            for j in range(a1+1, b1):
                labels_multiple[j]=1
            flag1=False
        elif(flag2):
            for j in range(a2+1, b2):
                labels_multiple[j]=2
            flag2=False
        elif(flag3):
            for j in range(a3+1, b3):
                labels_multiple[j]=3
            flag3=False
        elif(flag4):
            for j in range(a4+1, b4):
                labels_multiple[j]=4
            flag4=False
    
    if(a1>b1):
        for i in range(a1+1, N):
            labels_multiple[i]=1
    elif(a2>b2):
        for i in range(a2+1, N):
            labels_multiple[i]=2
    elif(a3>b3):
        for i in range(a3+1, N):
            labels_multiple[i]=3
    elif(a4>b4):
        for i in range(a4+1, N):
            labels_multiple[i]=4
    
    labels_multiple=labels_multiple[0:int(N/12)]

    
    np.savetxt(folder5+attribute_file+'.csv', labels_multiple, fmt='%d', header='Multiple')    

    
#     dataFile=full_path1[i]
#     #dataFile = './output/0808_00000_2457_Lau.mat'
#     data = scio.loadmat(dataFile)
#     acc_data_x=data['acc_data_x'][0]
#     acc_data_y=data['acc_data_y'][0]
#     acc_data_z=data['acc_data_z'][0]
#     N=len(acc_data_x)
#     print('dataFile %s. \nSize of acc_data_x: %d'%(dataFile, len(acc_data_x)))   

0808_00000_2457_Lau
4 119
130 231
239 309
0808_00001_2426_Lau
60 142
0808_00001_2449_Lau
5 56
66 78
0808_00002_2454_Lau
3 18
21 176
180 213
0808_00003_2427_Lau
2 31
0808_00007_2451_Lau
0808_00008_2426_Lau
2 49
0808_00009_2427_Lau
4 13
0808_00010_2454_Lau
98 169
0808_00010_2457_Lau
13 19
72 101
0810_00006_2427_Lau
26 32
38 215
0810_00008_2427_Lau
70 78
88 192
0810_00009_2427_Lau
5 160
0810_00012_2427_Lau
109 128
134 163
0810_2426-034023_2426_Lau
1 9
0812_00005_2427_Lau
1 83
0812_00009_2449_Lau
0812_00012_2436_Lau
145 155
0813_00018_2427_Lau
14 58
0813_00019_2428_Lau
0813_00024_2446_Lau
12 44
64 65
71 78
0813_00025_2451_Lau
0813_00026_2451_Lau
0813_00026_2457_Lau
0 33
0813_00027_2451_Lau
56 69
0813_00027_2457_Lau
0814_00000_2427_Lau
6 151
158 225
227 240
259 262
266 369
0814_00001_2426_Lau
0814_00002_2427_Lau
0814_00002_2451_Lau
2 56
185 192
277 372
0814_00002_2457_Lau
5 59
88 109
159 165
186 189
220 229
362 377
0814_00003_2451_Lau
0814_00003_2457_Lau
0817_00001_2427_Lau
1 46
0817_00002_

### Feeding or not

In [372]:
labels_feeding=np.zeros(N)
i=1
flag=False
for i in range(len(df)):
    if(behavior[i]=="Feeding" and status[i]=="START"): 
        a=math.floor(time[i])
#         if(a>int(N/12)):
#             break
        flag=False
        if(a+1-time[i]>=0.5):
            labels_feeding[a]=1
        else:
            labels_feeding[a]=0
    elif(behavior[i]=="Feeding" and status[i]=="STOP"):
        b=math.floor(time[i])
#         if(b>int(N/12)):
#             break
        flag=True
        if(time[i]-b>=0.5):
            labels_feeding[b]=1
        else:
            labels_feeding[b]=0
    if(flag):
        print(a, b)
        
        for j in range(a+1, b):
            labels_feeding[j]=1
        flag=False
if(a>b):
    for i in range(a+1, N):
        labels_feeding[i]=1
    

labels_feeding=labels_feeding[0:int(N/12)]


np.savetxt('./Label_Feeding/0808_00000_2457_Lau(2).csv', labels_feeding, fmt='%d')    
  

4 119
130 231
239 309


### classification

In [7]:
import scipy.io as scio
import pandas as pd

In [8]:
label_file=pd.read_csv(folder5+'0808_00000_2457_Lau'+'.csv')
label_file['# Multiple'][0]

5

In [11]:
folder6='./Multiple_labels/'
filename6=[]
full_path6=[]
for filename in os.listdir(folder6):
    filename6.append(filename)

In [13]:
#Read data
import scipy.io as scio
import pandas as pd

# dataFile = './output/0808_00000_2457_Lau.mat'
# data = scio.loadmat(dataFile)
# acc_data_x=data['acc_data_x'][0]
# acc_data_y=data['acc_data_y'][0]
# acc_data_z=data['acc_data_z'][0]
# N=len(acc_data_x)
# print('dataFile %s. \nSize of acc_data_x: %d'%(dataFile, len(acc_data_x)))


total_label=[]

#dataFile=full_path2[iteration]
attribute_file=filename6[0][0:len(filename6[0])-4]
dataFile=folder2+attribute_file+'.mat'
#print(dataFile)
#dataFile = './output/0808_00000_2457_Lau.mat'
data = scio.loadmat(dataFile)
acc_data_x=data['acc_data_x'][0]
acc_data_y=data['acc_data_y'][0]
acc_data_z=data['acc_data_z'][0]

NaN_number=[]
for j in range(len(acc_data_x)):
    if(math.isnan(acc_data_x[j]) or math.isnan(acc_data_y[j]) or math.isnan(acc_data_z[j])\
      or math.isinf(acc_data_x[j]) or math.isinf(acc_data_y[j]) or math.isinf(acc_data_z[j])):
        NaN_number.append(math.floor(j/12))
#print(NaN_number)
total_data=[acc_data_x, acc_data_y, acc_data_z]

label_file=pd.read_csv(folder6+attribute_file+'.csv')
for i in range(len(label_file['# Multiple'])):
    if(i in NaN_number): 
        total_label.append(5)
        continue
    total_label.append(label_file['# Multiple'][i])
    

for iteration in range(1,len(filename6)):
    attribute_file=filename6[iteration][0:len(filename6[iteration])-4]
    f=False
    for index in range(len(filename2)):
        if attribute_file.lower()==filename2[index][0:len(filename2[index])-4].lower():
            f=True
        if(f==False):
            continue

    #dataFile=full_path2[iteration]
    dataFile=folder2+attribute_file+'.mat'
    #dataFile = './output/0808_00000_2457_Lau.mat'
    data = scio.loadmat(dataFile)
    acc_data_x=data['acc_data_x'][0]
    acc_data_y=data['acc_data_y'][0]
    acc_data_z=data['acc_data_z'][0]
    
    NaN_number=[] #
    for j in range(len(acc_data_x)):
        if(math.isnan(acc_data_x[j]) or math.isnan(acc_data_y[j]) or math.isnan(acc_data_z[j])\
      or math.isinf(acc_data_x[j]) or math.isinf(acc_data_y[j]) or math.isinf(acc_data_z[j])):
            NaN_number.append(math.floor(j/12))
    # print(NaN_number)  
    a=[acc_data_x, acc_data_y, acc_data_z]
    total_data=np.concatenate((total_data, a), axis=1)
    
    label_file=pd.read_csv(folder6+attribute_file+'.csv')
    aa=[]
    for i in range(len(label_file['# Multiple'])):
        if(i in NaN_number): 
            aa.append(5)
            total_label.append(5)
            continue
        aa.append(label_file['# Multiple'][i])
        total_label.append(label_file['# Multiple'][i])

    print(len(a[0])/12, len(aa))


c=[]
for i in range(0,len(total_data[0]), 12):
    c.append(np.array([total_data[0][i:i+12], total_data[1][i:i+12], total_data[2][i:i+12]]))

d=[]
# c=d[0].reshape(1,36)
for i in range(len(c)):
    d.append(c[i].reshape(1, 36))
    d[i]=d[i][0]

dataset=d

print(len(dataset), len(total_label))      
        

# a=[];
# acc_x=np.zeros(N)
# acc_y=np.zeros(N)
# acc_z=np.zeros(N)
# for i in range(N):
#     acc_x[i]=int(acc_data_x[i])
#     acc_y[i]=int(acc_data_y[i])
#     acc_z[i]=int(acc_data_z[i])
    


142.0 142
142.0 142
213.0 213
41.0 41
41.0 41
53.0 53
33.0 33
283.0 283
283.0 283
269.0 269
234.0 234
161.0 161
269.0 269
60.0 60
148.0 148
209.0 209
532.0 532
64.0 64
20.0 20
95.0 95
36.0 36
69.0 69
69.0 69
119.0 119
119.0 119
402.0 402
21.0 21
396.0 396
396.0 396
396.0 396
938.0 938
938.0 938
52.0 52
234.0 234
234.0 234
330.0 330
330.0 330
84.0 84
538.0 538
174.0 174
65.0 65
114.0 114
832.0 832
832.0 832
832.0 832
196.0 196
56.0 56
292.0 292
1201.0 1201
1201.0 1201
625.0 625
625.0 625
625.0 625
625.0 625
297.0 297
297.0 297
297.0 297
201.0 201
207.0 207
552.0 552
276.0 276
446.0 446
183.0 183
123.0 123
1034.0 1034
656.0 656
656.0 656
656.0 656
111.0 111
399.0 399
265.0 265
67.0 67
153.0 153
383.0 383
383.0 383
290.0 290
180.0 180
215.0 215
215.0 215
104.0 104
31.0 31
137.0 137
782.0 782
169.0 169
492.0 492
74.0 74
215.0 215
196.0 196
167.0 167
28505 28505


In [14]:
dataset[0]

array([ 1884.,  1950.,  1898.,  1945.,  1954.,  2020.,  1942.,  2004.,
        1904.,  1984.,  1911.,  1943.,  2163.,  2280.,  2321.,  2300.,
        2286.,  2427.,  2260.,  2261.,  2330.,  2220.,  2214.,  2197.,
        2320.,  2287.,  2324.,  2242.,  2260.,  2272.,  2216.,  2268.,
        2339.,  2330.,  2280.,  2248.], dtype=float32)

### remove 5

In [15]:
new_dataset=[]
new_labels_multiple=[]
for i in range(len(dataset)):
    if(total_label[i]==5):
        continue
    new_dataset.append(dataset[i])
    new_labels_multiple.append(total_label[i])


In [16]:
print(len(new_dataset), len(new_labels_multiple))

20348 20348


In [846]:
math.isinf(5)

False

In [21]:
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm
from sklearn.model_selection import cross_val_score

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=10)

clf = clf.fit(, Y)

A=clf.predict(X_test)
A

### Multiple behaviors

In [18]:
clf = svm.SVC(kernel='rbf', C=1)
scores = cross_val_score(clf, new_dataset, new_labels_multiple, cv=10)

scores
# clf = svm.SVC(kernel='Linear', C=1).fit(X_train, y_train)
# clf.score(X_test, y_test)  

array([ 0.72888016,  0.72789784,  0.72789784,  0.69548134,  0.72888016,
        0.72825553,  0.72825553,  0.72861357,  0.72883858,  0.72883858])

In [19]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.73 (+/- 0.02)


In [ ]:
from sklearn.model_selection import ShuffleSplit
#n_samples = dataset.shape[0]
clf = svm.SVC(kernel='rbf', C=1)
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
scores2=cross_val_score(clf, new_dataset, new_labels_multiple, cv=cv)
scores2

In [590]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores2.mean(), scores2.std() * 2))

Accuracy: 0.63 (+/- 0.07)


In [425]:
from sklearn.cross_validation import KFold

//anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### Feeding or not

In [9]:
folder6='./Feeding/'
filename6=[]
full_path6=[]
for filename in os.listdir(folder6):
    filename6.append(filename)

In [10]:
#Read data
import scipy.io as scio
import pandas as pd

# dataFile = './output/0808_00000_2457_Lau.mat'
# data = scio.loadmat(dataFile)
# acc_data_x=data['acc_data_x'][0]
# acc_data_y=data['acc_data_y'][0]
# acc_data_z=data['acc_data_z'][0]
# N=len(acc_data_x)
# print('dataFile %s. \nSize of acc_data_x: %d'%(dataFile, len(acc_data_x)))


total_label=[]

#dataFile=full_path2[iteration]
attribute_file=filename6[0][0:len(filename6[0])-4]
dataFile=folder2+attribute_file+'.mat'
#print(dataFile)
#dataFile = './output/0808_00000_2457_Lau.mat'
data = scio.loadmat(dataFile)
acc_data_x=data['acc_data_x'][0]
acc_data_y=data['acc_data_y'][0]
acc_data_z=data['acc_data_z'][0]

NaN_number=[]
for j in range(len(acc_data_x)):
    if(math.isnan(acc_data_x[j]) or math.isnan(acc_data_y[j]) or math.isnan(acc_data_z[j])\
      or math.isinf(acc_data_x[j]) or math.isinf(acc_data_y[j]) or math.isinf(acc_data_z[j])):
        NaN_number.append(math.floor(j/12))
#print(NaN_number)
total_data=[acc_data_x, acc_data_y, acc_data_z]

label_file=pd.read_csv(folder6+attribute_file+'.csv')
for i in range(len(label_file['# Feeding'])):
    if(i in NaN_number): 
        total_label.append(5)
        continue
    total_label.append(label_file['# Feeding'][i])
    

for iteration in range(1,len(filename6)):
    attribute_file=filename6[iteration][0:len(filename6[iteration])-4]
    f=False
    for index in range(len(filename2)):
        if attribute_file.lower()==filename2[index][0:len(filename2[index])-4].lower():
            f=True
        if(f==False):
            continue

    #dataFile=full_path2[iteration]
    dataFile=folder2+attribute_file+'.mat'
    #dataFile = './output/0808_00000_2457_Lau.mat'
    data = scio.loadmat(dataFile)
    acc_data_x=data['acc_data_x'][0]
    acc_data_y=data['acc_data_y'][0]
    acc_data_z=data['acc_data_z'][0]
    
    NaN_number=[] #
    for j in range(len(acc_data_x)):
        if(math.isnan(acc_data_x[j]) or math.isnan(acc_data_y[j]) or math.isnan(acc_data_z[j])\
      or math.isinf(acc_data_x[j]) or math.isinf(acc_data_y[j]) or math.isinf(acc_data_z[j])):
            NaN_number.append(math.floor(j/12))
    # print(NaN_number)  
    a=[acc_data_x, acc_data_y, acc_data_z]
    total_data=np.concatenate((total_data, a), axis=1)
    
    label_file=pd.read_csv(folder6+attribute_file+'.csv')
    aa=[]
    for i in range(len(label_file['# Feeding'])):
        if(i in NaN_number): 
            aa.append(5)
            total_label.append(5)
            continue
        aa.append(label_file['# Feeding'][i])
        total_label.append(label_file['# Feeding'][i])

    print(len(a[0])/12, len(aa))


c=[]
for i in range(0,len(total_data[0]), 12):
    c.append(np.array([total_data[0][i:i+12], total_data[1][i:i+12], total_data[2][i:i+12]]))

d=[]
# c=d[0].reshape(1,36)
for i in range(len(c)):
    d.append(c[i].reshape(1, 36))
    d[i]=d[i][0]

dataset=d

print(len(dataset), len(total_label))      
        

# a=[];
# acc_x=np.zeros(N)
# acc_y=np.zeros(N)
# acc_z=np.zeros(N)
# for i in range(N):
#     acc_x[i]=int(acc_data_x[i])
#     acc_y[i]=int(acc_data_y[i])
#     acc_z[i]=int(acc_data_z[i])
    


142.0 142
142.0 142
213.0 213
41.0 41
41.0 41
53.0 53
33.0 33
283.0 283
283.0 283
269.0 269
234.0 234
161.0 161
269.0 269
60.0 60
148.0 148
209.0 209
532.0 532
64.0 64
20.0 20
95.0 95
36.0 36
69.0 69
69.0 69
119.0 119
119.0 119
402.0 402
21.0 21
396.0 396
396.0 396
396.0 396
938.0 938
938.0 938
52.0 52
234.0 234
234.0 234
330.0 330
330.0 330
84.0 84
538.0 538
174.0 174
65.0 65
114.0 114
832.0 832
832.0 832
832.0 832
196.0 196
56.0 56
292.0 292
1201.0 1201
1201.0 1201
625.0 625
625.0 625
625.0 625
625.0 625
297.0 297
297.0 297
297.0 297
201.0 201
207.0 207
552.0 552
276.0 276
446.0 446
183.0 183
123.0 123
1034.0 1034
656.0 656
656.0 656
656.0 656
111.0 111
399.0 399
265.0 265
67.0 67
153.0 153
383.0 383
383.0 383
290.0 290
180.0 180
215.0 215
215.0 215
104.0 104
31.0 31
137.0 137
782.0 782
169.0 169
492.0 492
74.0 74
215.0 215
196.0 196
167.0 167
28505 28505


In [11]:
new_dataset=[]
new_labels_multiple=[]
for i in range(len(dataset)):
    if(total_label[i]==5):
        continue
    new_dataset.append(dataset[i])
    new_labels_multiple.append(total_label[i])


In [14]:
clf = svm.SVC(kernel='rbf', C=1)
scores = cross_val_score(clf, new_dataset, new_labels_multiple, cv=10)

scores
# clf = svm.SVC(kernel='Linear', C=1).fit(X_train, y_train)
# clf.score(X_test, y_test)  

array([ 0.57361163,  0.58762118,  0.59567326,  0.58373741,  0.58784036,
        0.58784036,  0.58784036,  0.54755688,  0.58784036,  0.58784036])

In [15]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.58 (+/- 0.03)


### Multiple features

In [191]:
folder6='./Feeding/'
filename6=[]
full_path6=[]
for filename in os.listdir(folder6):
    filename6.append(filename)

In [120]:
import numpy as np
import scipy
from astropy.stats import median_absolute_deviation

In [199]:
#Read data
import scipy.io as scio
import pandas as pd

# dataFile = './output/0808_00000_2457_Lau.mat'
# data = scio.loadmat(dataFile)
# acc_data_x=data['acc_data_x'][0]
# acc_data_y=data['acc_data_y'][0]
# acc_data_z=data['acc_data_z'][0]
# N=len(acc_data_x)
# print('dataFile %s. \nSize of acc_data_x: %d'%(dataFile, len(acc_data_x)))


total_label=[]

#dataFile=full_path2[iteration]
attribute_file=filename6[0][0:len(filename6[0])-4]
dataFile=folder2+attribute_file+'.mat'
#print(dataFile)
#dataFile = './output/0808_00000_2457_Lau.mat'
data = scio.loadmat(dataFile)
acc_data_x=data['acc_data_x'][0]
acc_data_y=data['acc_data_y'][0]
acc_data_z=data['acc_data_z'][0]

NaN_number=[]
for j in range(len(acc_data_x)):
    if(math.isnan(acc_data_x[j]) or math.isnan(acc_data_y[j]) or math.isnan(acc_data_z[j])\
      or math.isinf(acc_data_x[j]) or math.isinf(acc_data_y[j]) or math.isinf(acc_data_z[j])):
        NaN_number.append(math.floor(j/12))
#print(NaN_number)
total_data=[acc_data_x, acc_data_y, acc_data_z]

label_file=pd.read_csv(folder6+attribute_file.lower()+'.csv')
for i in range(len(label_file['# Feeding'])):
    if(i in NaN_number): 
        total_label.append(5)
        continue
    total_label.append(label_file['# Feeding'][i])
    

for iteration in range(1,11):
    attribute_file=filename6[iteration][0:len(filename6[iteration])-4]
    f=False
    for index in range(len(filename2)):
        if attribute_file.lower()==filename2[index][0:len(filename2[index])-4].lower():
            f=True
    if(f==False):
        continue

    #dataFile=full_path2[iteration]
    dataFile=folder2+attribute_file+'.mat'
    #dataFile = './output/0808_00000_2457_Lau.mat'
    data = scio.loadmat(dataFile)
    acc_data_x=data['acc_data_x'][0]
    acc_data_y=data['acc_data_y'][0]
    acc_data_z=data['acc_data_z'][0]
    
    NaN_number=[] #
    for j in range(len(acc_data_x)):
        if(math.isnan(acc_data_x[j]) or math.isnan(acc_data_y[j]) or math.isnan(acc_data_z[j])\
      or math.isinf(acc_data_x[j]) or math.isinf(acc_data_y[j]) or math.isinf(acc_data_z[j])):
            NaN_number.append(math.floor(j/12))
    # print(NaN_number)  
    a=[acc_data_x, acc_data_y, acc_data_z]
    total_data=np.concatenate((total_data, a), axis=1)
    
    label_file=pd.read_csv(folder6+attribute_file.lower()+'.csv')
    aa=[]
    for i in range(len(label_file['# Feeding'])):
        if(i in NaN_number): 
            aa.append(5)
            total_label.append(5)
            continue
        aa.append(label_file['# Feeding'][i])
        total_label.append(label_file['# Feeding'][i])

    print(len(a[0])/12, len(aa))


c=[]
for i in range(0,len(total_data[0]), 12):
    a0=total_data[0][i:i+12]
    b0=total_data[1][i:i+12]
    c0=total_data[2][i:i+12]
    a0_var=np.var(a0)
    b0_var=np.var(b0)
    c0_var=np.var(c0)
    a0_deviation=median_absolute_deviation(a0)
    b0_deviation=median_absolute_deviation(b0)
    c0_deviation=median_absolute_deviation(c0)
#     if(a0_var>10000):
#         a0_var/=5
#     if(b0_var>10000):
#         b0_var/=5
#     if(c0_var>10000):
#         c0_var/=5
#     if(a0_deviation<500):
#         a0_deviation*=5
#     if(b0_deviation<500):
#         b0_deviation*=5
#     if(a0_deviation<500):
#         b0_deviation*=5
    
    a0_1=[ np.mean(a0), np.median(a0), a0_var, np.max(a0),np.min(a0), np.percentile(a0,25), np.percentile(a0,50), np.percentile(a0,75),a0_deviation]
    b0_1=[ np.mean(b0), np.median(b0),  b0_var, np.max(b0), np.min(b0), np.percentile(b0,25), np.percentile(a0,50), np.percentile(b0,75),a0_deviation]
    c0_1=[ np.mean(c0), np.median(c0),  c0_var, np.max(c0), np.min(c0), np.percentile(c0,25), np.percentile(a0,50), np.percentile(c0,75),a0_deviation]
    
    a0_feature=total_data[0][i:i+12]
    b0_feature=total_data[1][i:i+12]
    c0_feature=total_data[2][i:i+12]
    
    a0_feature=np.concatenate((a0_feature,a0_1))
    b0_feature=np.concatenate((b0_feature,b0_1))
    c0_feature=np.concatenate((c0_feature,c0_1))
    
    #  c.append(np.array([total_data[0][i:i+12], total_data[1][i:i+12], total_data[2][i:i+12]]))
    c.append(np.array([a0_feature, b0_feature, c0_feature]))
d=[]
# c=d[0].reshape(1,60) (12+8)*3
for i in range(len(c)):
    d.append(c[i].reshape(1, 63))
    d[i]=d[i][0]

dataset=d


print(len(dataset), len(total_label))      
        

# a=[];
# acc_x=np.zeros(N)
# acc_y=np.zeros(N)
# acc_z=np.zeros(N)
# for i in range(N):
#     acc_x[i]=int(acc_data_x[i])
#     acc_y[i]=int(acc_data_y[i])
#     acc_z[i]=int(acc_data_z[i])
    


142.0 142
142.0 142
213.0 213
41.0 41
41.0 41
53.0 53
33.0 33
283.0 283
283.0 283
269.0 269


//anaconda/lib/python3.6/site-packages/numpy/lib/function_base.py:3858: RuntimeWarning: Invalid value encountered in median
  r = func(a, **kwargs)
//anaconda/lib/python3.6/site-packages/numpy/lib/function_base.py:4116: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


1808 1808


In [83]:
d=[]
# c=d[0].reshape(1,60) (12+8)*3
for i in range(len(c)):
    d.append(c[i].reshape(1, 54))
    d[i]=d[i][0]

dataset=d


print(len(dataset), len(total_label))      
        

ValueError: cannot reshape array of size 63 into shape (1,54)

In [200]:
dataset[0]

array([ 1884.        ,  1950.        ,  1898.        ,  1945.        ,
        1954.        ,  2020.        ,  1942.        ,  2004.        ,
        1904.        ,  1984.        ,  1911.        ,  1943.        ,
        1944.91662598,  1944.        ,  1626.07653809,  2020.        ,
        1884.        ,  1909.25      ,  1944.        ,  1961.5       ,
          36.5       ,  2163.        ,  2280.        ,  2321.        ,
        2300.        ,  2286.        ,  2427.        ,  2260.        ,
        2261.        ,  2330.        ,  2220.        ,  2214.        ,
        2197.        ,  2271.58325195,  2270.5       ,  4555.90966797,
        2427.        ,  2163.        ,  2218.5       ,  1944.        ,
        2305.25      ,    36.5       ,  2320.        ,  2287.        ,
        2324.        ,  2242.        ,  2260.        ,  2272.        ,
        2216.        ,  2268.        ,  2339.        ,  2330.        ,
        2280.        ,  2248.        ,  2282.16674805,  2276.        ,
      

In [201]:
new_dataset=[]
new_labels_multiple=[]
for i in range(len(dataset)):
    if(total_label[i]==5):
        continue
    new_dataset.append(dataset[i])
    new_labels_multiple.append(total_label[i])

In [202]:
len(new_dataset)
len(new_labels_multiple)

1785

In [203]:
clf = svm.SVC(kernel='rbf', C=1)
scores = cross_val_score(clf, new_dataset, new_labels_multiple, cv=10)

scores
# clf = svm.SVC(kernel='Linear', C=1).fit(X_train, y_train)
# clf.score(X_test, y_test)  

array([ 0.55865922,  0.55865922,  0.55865922,  0.55865922,  0.55865922,
        0.55865922,  0.55865922,  0.56179775,  0.55932203,  0.55932203])

In [204]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.56 (+/- 0.00)


In [83]:
dataset[0]

array([ 1944.91662598,  1944.        ,  2020.        ,  1884.        ,
        1626.07653809,  1909.25      ,  1961.5       ,    36.5       ,
        2271.58325195,  2270.5       ,  2427.        ,  2163.        ,
        4555.90966797,  2218.5       ,  2305.25      ,    50.5       ,
        2282.16674805,  2276.        ,  2339.        ,  2216.        ,
        1390.13879395,  2257.        ,  2321.        ,    31.        ])

In [78]:
np.percentile(dataset[0],75)

2277.5416870117188

In [7]:
import numpy as np
import scipy
from astropy.stats import median_absolute_deviation
np.var([1,2,3, 4, 5])
np.max([1,2,3,4,5])
a=[1,1,1,1,1]

b=median_absolute_deviation(a)
b

0.0

In [62]:
a=new_dataset[100]
a

array([ 1970.,  1996.,  1945.,  1964.,  1962.,  1932.,  1909.,  1988.,
        1924.,  1974.,  1982.,  1972.,  2192.,  2196.,  2198.,  2176.,
        2181.,  2192.,  2168.,  2140.,  2202.,  2129.,  2260.,  2165.,
        2444.,  2341.,  2308.,  2336.,  2320.,  2362.,  2338.,  2339.,
        2357.,  2355.,  2352.,  2339.], dtype=float32)

In [63]:
print(np.mean(a), np.median(a), np.max(a), np.min(a), np.var(a), np.percentile(a,25), np.percentile(a,75), \
median_absolute_deviation(a))

2164.11 2186.5 2444.0 1909.0 26360.7 1980.0 2336.5 153.5


In [170]:
signal = np.array([-2, 8, 6, 4, 1, 0, 3, 5], dtype=float)
fourier = np.fft.fft(signal)
fourier

array([ 25.00000000+0.j        ,   3.36396103-7.94974747j,
       -10.00000000+1.j        ,  -9.36396103-1.94974747j,
        -9.00000000+0.j        ,  -9.36396103+1.94974747j,
       -10.00000000-1.j        ,   3.36396103+7.94974747j])

In [205]:
attribute_file='0808_00001_2449_Lau'
#0808_00000_2457_Lau
#0808_00001_2449_Lau 0.81
#0808_00002_2454_Lau.csv 0.60

total_label=[]
#dataFile=full_path2[iteration]
#attribute_file=filename6[0][0:len(filename6[0])-4]
dataFile=folder2+attribute_file+'.mat'
#print(dataFile)
#dataFile = './output/0808_00000_2457_Lau.mat'
data = scio.loadmat(dataFile)
acc_data_x=data['acc_data_x'][0]
acc_data_y=data['acc_data_y'][0]
acc_data_z=data['acc_data_z'][0]

NaN_number=[]
for j in range(len(acc_data_x)):
    if(math.isnan(acc_data_x[j]) or math.isnan(acc_data_y[j]) or math.isnan(acc_data_z[j])\
      or math.isinf(acc_data_x[j]) or math.isinf(acc_data_y[j]) or math.isinf(acc_data_z[j])):
        NaN_number.append(math.floor(j/12))
#print(NaN_number)
total_data=[acc_data_x, acc_data_y, acc_data_z]

c=[]
for i in range(0,len(total_data[0]), 12):
    a0=total_data[0][i:i+12]
    b0=total_data[1][i:i+12]
    c0=total_data[2][i:i+12]
    a0_var=np.var(a0)
    b0_var=np.var(b0)
    c0_var=np.var(c0)
    a0_deviation=median_absolute_deviation(a0)
    b0_deviation=median_absolute_deviation(b0)
    c0_deviation=median_absolute_deviation(c0)
#     if(a0_var>10000):
#         a0_var/=5
#     if(b0_var>10000):
#         b0_var/=5
#     if(c0_var>10000):
#         c0_var/=5
#     if(a0_deviation<500):
#         a0_deviation*=5
#     if(b0_deviation<500):
#         b0_deviation*=5
#     if(a0_deviation<500):
#         b0_deviation*=5
    
    a0_1=[ np.mean(a0), np.median(a0), a0_var, np.max(a0),np.min(a0), np.percentile(a0,25), np.percentile(a0,50), np.percentile(a0,75),a0_deviation]
    b0_1=[ np.mean(b0), np.median(b0),  b0_var, np.max(b0), np.min(b0), np.percentile(b0,25), np.percentile(a0,50), np.percentile(b0,75),a0_deviation]
    c0_1=[ np.mean(c0), np.median(c0),  c0_var, np.max(c0), np.min(c0), np.percentile(c0,25), np.percentile(a0,50), np.percentile(c0,75),a0_deviation]
    
    a0_feature=total_data[0][i:i+12]
    b0_feature=total_data[1][i:i+12]
    c0_feature=total_data[2][i:i+12]
    
    a0_feature=np.concatenate((a0_feature,a0_1))
    b0_feature=np.concatenate((b0_feature,b0_1))
    c0_feature=np.concatenate((c0_feature,c0_1))
    
    #  c.append(np.array([total_data[0][i:i+12], total_data[1][i:i+12], total_data[2][i:i+12]]))
    c.append(np.array([a0_feature, b0_feature, c0_feature]))
d=[]
# c=d[0].reshape(1,60) (12+8)*3
for i in range(len(c)):
    d.append(c[i].reshape(1, 63))
    d[i]=d[i][0]

dataset=d


print(len(dataset), len(total_label))    

142 0


In [206]:
dataset[0]

array([ 2130.        ,  2021.        ,  2142.        ,  2165.        ,
        2113.        ,  2123.        ,  2077.        ,  2070.        ,
        2049.        ,  2038.        ,  2048.        ,  2088.        ,
        2088.66674805,  2082.5       ,  1917.38903809,  2165.        ,
        2021.        ,  2048.75      ,  2082.5       ,  2124.75      ,
          37.5       ,  2260.        ,  2241.        ,  2259.        ,
        2251.        ,  2289.        ,  2277.        ,  2332.        ,
        2301.        ,  2325.        ,  2331.        ,  2286.        ,
        2280.        ,  2286.        ,  2283.        ,   889.        ,
        2332.        ,  2241.        ,  2259.75      ,  2082.5       ,
        2307.        ,    37.5       ,  1601.        ,  1498.        ,
        1613.        ,  1641.        ,  1636.        ,  1582.        ,
        1612.        ,  1667.        ,  1656.        ,  1689.        ,
        1653.        ,  1581.        ,  1619.08337402,  1624.5       ,
      

In [207]:
total_label=[]

label_file=pd.read_csv(folder6+attribute_file.lower()+'.csv')
for i in range(len(label_file['# Feeding'])):
    if(i in NaN_number): 
        total_label.append(5)
        continue
    total_label.append(label_file['# Feeding'][i])

In [208]:
len(total_label)

142

In [209]:
new_dataset=[]
new_labels_multiple=[]
for i in range(len(dataset)):
    if(total_label[i]==5):
        continue
    new_dataset.append(dataset[i])
    new_labels_multiple.append(total_label[i])

In [213]:
len(new_dataset)
# dd=0
# for i in new_labels_multiple:
#     if(i==1):
#         dd+=1
# print(dd/555)
        

142

In [214]:
clf = svm.SVC(kernel='rbf', C=1)
scores = cross_val_score(clf, new_dataset, new_labels_multiple, cv=10)

scores
# clf = svm.SVC(kernel='Linear', C=1).fit(X_train, y_train)
# clf.score(X_test, y_test)  

array([ 0.5625    ,  0.57142857,  0.57142857,  0.57142857,  0.57142857,
        0.57142857,  0.57142857,  0.57142857,  0.57142857,  0.57142857])

In [215]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.57 (+/- 0.01)


In [181]:
#0814_00003_2451_Lau.csv
print(folder6)

./Multiple_labels/


In [185]:
 X_train, X_test, y_train, y_test = train_test_split(
...     new_dataset, new_labels_multiple, test_size=0.3, random_state=0)



clf = svm.SVC(kernel='rbf', C=1).fit(X_train, y_train)
clf.score(X_test, y_test)

0.89221556886227549

In [187]:
from sklearn.model_selection import ShuffleSplit
#n_samples = iris.data.shape[0]
cv = ShuffleSplit(n_splits=10, test_size=0.3, random_state=0)
s=cross_val_score(clf,new_dataset, new_labels_multiple, cv=cv)
s

array([ 0.89221557,  0.94011976,  0.90419162,  0.95209581,  0.94011976,
        0.91616766,  0.95808383,  0.91616766,  0.92814371,  0.92814371])